# Confidence Interval for Two Independent Means

---

## Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.power import TTestIndPower

plt.style.use("fivethirtyeight")
%matplotlib inline

In [2]:
def sample_creator(n, mean, std):
    sample = stats.norm.rvs(loc=0, scale=std, size=n)
    sample = sample - np.mean(sample)
    sample = sample * (std/sample.std(ddof=1))
    return sample + mean

---

Per fare inferenza sulla differenza fra due medie indipendenti, ci sono delle condizioni da rispettare:

- **Independence within groups**: Le osservazioni raccolte nei *sample* devono essere indipendenti. E' molto probabile che si verifichi nel caso in cui sia stato applicato il *random sampling* (in un *observational study*) o il *random assignment* (in un *experiment*). Nel caso di *random sampling* senza *replacement*, i due *sample size* $n_1$ e $n_2$ devono essere entrambi inferiori rispetto al 10% della popolazione.


- **Independence between groups**: I gruppi devono essere indipendenti fra loro, cioè le osservazioni al loro interno non devono essere *paired*  fra un gruppo e l'altro.


- **Sample size/skew**: Maggiore skewness necessita di *sample size* altrettanto alti.


- **Population distribution**: la popolazione deve seguire andamento quasi normale.

Per costruire un *confidence interval* che stimi la differenza fra **two independent means**, la *point estimate* è la differenza fra le due medie, mentre lo *standard error* va calcolato come la radice quadrata della somma delle varianze dei due gruppi divise per il relativo *sample size*. Infine, i *degrees of freedom* della *t-distribution* in questo caso sono pari al minimo fra i due *sample size* meno uno. Questa è la cosiddetta **Welch Procedure**.<br><br>

$$\large CI: (\bar{x_1}-\bar{x_2})\pm t_{df}^*SE_{(\bar{x_1}-\bar{x_2})}$$

$$\large SE_{(\bar{x_1}-\bar{x_2})}=
\sqrt{\frac{s_1^2}{n_1}+\frac{s_2^2}{n_2}}$$

$$\large df=min(n_1-1,\; n_2-1)$$

In realtà, la formula originale per il calcolo dei *df* è:

$$\large df=\frac{\left(\frac{s_1^2}{n_1}+\frac{s_2^2}{n_2}\right)^2}
{\frac{1}{n_1-1}\left(\frac{s_1^2}{n_1}\right)^2+
\frac{1}{n_2-1}\left(\frac{s_2^2}{n_2}\right)^2}$$

---

La **pooled variance** procedure è una tecnica utilizzata per sopperire al fatto che la *sampling distribution* nata dalla differenza fra le due *sample means* non è perfettamente allineata con la *t-distribution*.

L'assunzione di fondo è che le varianze delle due popolazioni siano coincidenti.

$$\large \sigma_1^2=\sigma_2^2=\sigma^2$$

Possiamo costruire un *estimator* della *common population variance* con la cosiddetta **pooled sample variance**:

$$\large s_p^2=\frac{(n_1-1)s_1^2+(n_2-1)s_2^2}{n_1+n_2-2}$$

Lo **standard error** della differenza di *sample means* dev'essere costruito come segue:

$$\large SE(\bar{X_1}-\bar{X_2})=s_p\sqrt{\frac{1}{n_1}+\frac{1}{n_2}}$$

Questo *standard error* assume il ruolo di *estimator* della vera deviazione standard della *sampling distribution* della differenza fra le due medie.

Ora possiamo costruire il *confidence interval*:

$$\large \bar{X_1}-\bar{X_2}\pm t_{\alpha/2}\cdot SE(\bar{X_1}-\bar{X_2})$$

I *df* da associare al *critical t-score* sono gli stessi della *pooled variance*:

$$\large df=n_1-n_2-2$$

---

**\[Esempio\]** Un sample di 22 soggetti ha pranzato mentre giocava al pc. Sono stati calcolati i grammi di biscotti consumati come snack in seguito al pranzo. La sample mean è 52.1g, con deviazione standard 45.1g. Chi non ha giocato al pc, invece, ha consumato in media 27.1g di biscotti, con una deviazione standard di 26.4g. Anche in questo caso il sample size è 22. Calcolare un 95% CI relativo alla differenza in media di biscotti assunti dopo pranzo dai due gruppi.

In [3]:
def confint_two_mean_diff(cl, sample_1, sample_2):
    df = min(len(sample_1) - 1, len(sample_2) - 1)
    crit_value = stats.t.ppf(q = 1- ((1 - cl)/2), df = df)
    point_estimate = sample_1.mean() - sample_2.mean()
    std_error = np.sqrt(
                    np.power(sample_1.std(ddof=1), 2)/len(sample_1) +
                    np.power(sample_2.std(ddof=1), 2)/len(sample_2))
    lower_bound = point_estimate - crit_value * std_error
    upper_bound = point_estimate + crit_value * std_error
    return lower_bound, upper_bound

In [4]:
n_1 = 22
n_2 = 22
xbar_1 = 52.1
xbar_2 = 27.1
std_1 = 45.1
std_2 = 26.4

sample_1 = sample_creator(n_1, xbar_1, std_1)
sample_2 = sample_creator(n_2, xbar_2, std_2)

print(sample_1.mean(), sample_1.std(ddof = 1))
print(sample_2.mean(), sample_2.std(ddof = 1))

52.1 45.10000000000001
27.1 26.399999999999995


In [5]:
confint_two_mean_diff(0.95, sample_1, sample_2)

(1.8297977714906857, 48.17020222850931)

---

**[Esempio]** Ad un test per il PTSD, molti mentono. Abbiamo 49 veterani che cercano solo compensation e 70 che cercano un trattamento. Vogliamo capire se c'è differenza fra gli score medi dei due gruppi. Nel gruppo compensation abbiamo una sample mean di 9.76 e dev. std. di 4.90, mentre nel gruppo treatment abbiamo una sample mean di 6.48 e una dev. std. di 3.49.

In [6]:
n_1 = 49
xbar_1 = 9.76
s_1 = 4.90

n_2 = 70
xbar_2 = 6.48
s_2 = 3.49

sample_1 = sample_creator(n_1, xbar_1, s_1)
sample_2 = sample_creator(n_2, xbar_2, s_2)

print(sample_1.mean(), sample_1.std(ddof=1))
print(sample_2.mean(), sample_2.std(ddof=1))

9.76 4.8999999999999995
6.4799999999999995 3.49


In [7]:
def confint_two_means_pooled(sample_1, sample_2, alpha=0.05):
    n_1, n_2 = len(sample_1), len(sample_2)
    pooled_variance = ((n_1-1)*np.var(sample_1,ddof=1)+\
                       (n_2-1)*np.var(sample_2, ddof = 1))\
                        /(n_1+n_2-2)
    std_error = np.sqrt(pooled_variance)*np.sqrt((1/n_1)+(1/n_2))
    critical_t = stats.t.ppf(q = 1-(alpha/2), df = n_1+n_2-2)
    point_estimate = sample_1.mean() - sample_2.mean()
    margin_error = critical_t * std_error
    return (point_estimate - margin_error, point_estimate + margin_error)

In [8]:
confint_two_means_pooled(sample_1, sample_2)

(1.7575594214519832, 4.802440578548017)

Siamo sicuri al 95% che la differenza tra la media del primo gruppo e quella del secondo sia contenuta in questo intervallo. 

Per via del fatto che l'intervallo generato è strettamente maggiore di zero, possiamo affermare che in media gli score delle persone in cerca di trattamento sono inferiori rispetto agli score di chi cerca solo compensation mentendo. 

---

**[Esempio]** Si vuole studiare l'efficacia di un antidoto per veleno di serpente. Lo si testa sui maiali, creando un tratment group da 9 soggetti e un placebo group da 8 soggetti. La variabile da monitorare sarà la modifica in volume, per capire se l'antidoto sia efficace contro il gonfiore. 

I dati sono i seguenti, dove il gruppo 1 è il treatment, mentre il gruppo 2 è il placebo.

In [9]:
xbar_1 = 203.33
s_1 = 56.18
n_1 = 9

xbar_2 = 201.25
s_2 = 112.62
n_2 = 8

sample_1 = sample_creator(n_1, xbar_1, s_1)
sample_2 = sample_creator(n_2, xbar_2, s_2)

In [10]:
def confint_two_means_welch(sample_1, sample_2, alpha = 0.05):
    var_1, var_2 = np.var(sample_1,ddof=1), np.var(sample_2,ddof=1)
    n_1, n_2 = len(sample_1), len(sample_2)
    std_error = np.sqrt((var_1/n_1) + (var_2/n_2))
    point_estimate = np.mean(sample_1) - np.mean(sample_2)
    
    num_df = ((var_1/n_1)+(var_2/n_2))**2
    den_df = ((1/(n_1-1))*((var_1/n_1)**2))\
                + (((1/(n_2-1)))*((var_2/n_2)**2))
    df = num_df/den_df
    critical_t = stats.t.ppf(q = 1 - (alpha/2), df=df)
    margin_error = critical_t * std_error
    return (point_estimate-margin_error, point_estimate+margin_error)

In [11]:
confint_two_means_welch(sample_1, sample_2)

(-95.94636525583653, 100.10636525583661)

Zero è contenuto nell'intervallo, quindi è un valore possibile per la differenza di *population mean*. Questo vuol dire che non abbiamo alcuna certezza sulla differenza di media fra i due gruppi. 

---